# lecture

## 6.1 multinomial

- multinomial classification : 일반적인 binary classification에서 여러개의 함수로 늘어난 경우이다. 2개씩 비교하여 각 경우의 binary classification을 실행하여 결과를 종합한다.
- multinomial의 경우 계산할 때 matrix가 여러개의 행렬 형태이다.
- soft max : 결과로 나오는 예측값은 0~1사이로 표현되어야 다루기 편리하다. 이 때 다루는 것이 soft max이다. 나온 값들 중에서 선택된 것만 1로 바꾸고 나머지를 0으로 하는  
'ONE-HOT ENCODING'을 이용하면 결과값을 0과 1로 표현할 수 있다.
- Cross-entropy cost function : D(S,L) = (-(결과값) X log(예측값))의 모든 합

(S: 예측값, L : 결과값)
- 실제로는 앞에서 했던 ylog(H(x)) - (1-y)log(1-H(x))와 같은 의미이다.


# programming

## 6.2 softmax classification의 구현

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
x_data = [[1,2,1,1], [2,1,3,2], [3,1,3,4], [4,1,5,5], [1,7,5,5],[1,2,5,6],[1,6,6,6],[1,7,7,7]]
y_data = [[0,0,1],[0,0,1],[0,0,1],[0,1,0],[0,1,0],[0,1,0],[1,0,0],[1,0,0]]

X = tf.placeholder(tf.float32, shape = [None, 4])
Y = tf.placeholder(tf.float32, shape = [None, 3])
nb_classes=3  # 나가는 y의 값
w = tf.Variable(tf.random_normal([4,nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

#tf.nn.softmax computes softmax activations
#softmax = exp(logitx)/ reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, w) + b)

# cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate= 0.1).minimize(cost)

#launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict = {X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
            


0 13.0852
200 0.631112
400 0.521085
600 0.426783
800 0.335734
1000 0.253221
1200 0.224849
1400 0.204875
1600 0.188044
1800 0.173673
2000 0.161266


In [ ]:
# testing & one-hot encoding
a= sess.run(hypothesis, feed_dict={X: [[1,11,7,9]]})  
print(a, sess.run(tf.arg_max(a,1))) # arg_max 로 가장 큰 값을 골라줌


all = sess.run(hypothesis, feed_dict= {X: [[1,11,7,9],
                                           [1,3,4,3],
                                           [1,1,0,1]]})
print(all, sess.run(tf.arg_max(all, 1)))  # 한줄씩 큰값을 선정해서 자리번째 [1 0 2]를 출력해줌

## 6.3 Fancy Softmax classifier
- cross_entropy, one_hot, reshape이용

- Animal clssification : 동물들을 특징에 따라서 분류한다.

In [3]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

nb_classes = 7 

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape", Y_one_hot)
#자료값의 범위를 맞추기 위해 형식을 다시 함

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                 labels=tf.stop_gradient([Y_one_hot]))
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)#예측값 설정
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) # one_hot과 비교
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                 X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))

    # 학습을 마치고 결과값 출력
    pred = sess.run(prediction, feed_dict={X: x_data})
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

(101, 16) (101, 1)
one_hot Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape Tensor("Reshape:0", shape=(?, 7), dtype=float32)
Step:     0	Loss: 8.437	Acc: 1.98%
Step:   100	Loss: 0.607	Acc: 83.17%
Step:   200	Loss: 0.395	Acc: 92.08%
Step:   300	Loss: 0.306	Acc: 92.08%
Step:   400	Loss: 0.252	Acc: 93.07%
Step:   500	Loss: 0.215	Acc: 95.05%
Step:   600	Loss: 0.186	Acc: 95.05%
Step:   700	Loss: 0.164	Acc: 95.05%
Step:   800	Loss: 0.146	Acc: 95.05%
Step:   900	Loss: 0.131	Acc: 97.03%
Step:  1000	Loss: 0.119	Acc: 97.03%
Step:  1100	Loss: 0.109	Acc: 97.03%
Step:  1200	Loss: 0.100	Acc: 98.02%
Step:  1300	Loss: 0.092	Acc: 98.02%
Step:  1400	Loss: 0.086	Acc: 100.00%
Step:  1500	Loss: 0.080	Acc: 100.00%
Step:  1600	Loss: 0.075	Acc: 100.00%
Step:  1700	Loss: 0.070	Acc: 100.00%
Step:  1800	Loss: 0.066	Acc: 100.00%
Step:  1900	Loss: 0.063	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Predicti